In [1]:
import numpy as np
from scipy.linalg import svd, qr, solve_triangular

# Bonus task 13: gelss (easy)
Implement `gelss` (with numpy, python and scipy.svd).

Computes the minimum norm solution to a real linear least squares problem using the singular value decomposition (SVD).

In [2]:
def gelss(A, B, rcond=1e-10):
    
    A = np.asarray(A)
    B = np.asarray(B)
    m, n = A.shape
    
    # Ensure B is a 2D array for consistency
    if B.ndim == 1:
        B = B[:, np.newaxis]
    
    # Perform SVD
    U, s, V = svd(A, full_matrices=False)

    # Compute the optimal rank
    tol = rcond * max(s)
    rank = np.sum(s > tol)
    
    # Invert singular values with the rank cutoff
    s_inv = np.zeros_like(s)
    s_inv[:rank] = 1 / s[:rank]
    
    # Solve the least-squares problem
    X = V.T @ np.diag(s_inv) @ U.T @ B
    
    # Compute residuals if the system is overdetermined
    if m > n:
        residuals = np.linalg.norm(B - A @ X, axis=0)**2
    else:
        residuals = np.array([])
    
    # Return X as a 1D array if B was 1D
    if X.shape[1] == 1:
        X = X.ravel()
    
    return X, residuals, rank, s


In [3]:
# Example
A = np.array([[4, 2], [5, 4], [3, 7]])
B = np.array([2, 1, 2])
X, residuals, rank, singular_values = gelss(A, B)

print("A:\n", A, "\nB:\n", B)
print("X:", X)
print("Residuals:", residuals)
print("Rank:", rank)
print("Singular Values:", singular_values)

A:
 [[4 2]
 [5 4]
 [3 7]] 
B:
 [2 1 2]
X: [0.2221163  0.16110582]
Residuals: [1.23546235]
Rank: 2
Singular Values: [10.46003935  3.09638123]


# Bonus task 14: gelsd
Implement `gelsd` (with numpy and python ans scipy.svd). 

The problem is solved in three steps:

- (1) Reduce the coefficient matrix A to bidiagonal form with
     Householder transformations, reducing the original problem
     into a "bidiagonal least squares problem" (BLS)
- (2) Solve the BLS using a divide and conquer approach.
- (3) Apply back all the Householder transformations to solve
     the original least squares problem.

In [4]:
def gelsd(A, b, rcond=1e-10):
    
    # Perform SVD
    U, s, V = svd(A, full_matrices=False)

    # Compute the optimal rank
    tol = s[0] * rcond
    rank = np.sum(s > tol)

    # Compute the pseudo-inverse of singular values
    s_inv = np.zeros_like(s)
    s_inv[:rank] = 1 / s[:rank]

    # Solve for x
    X = V.T @ (s_inv * (U.T @ b))

    # Compute residuals if the system is overdetermined
    if A.shape[0] > A.shape[1] and rank == A.shape[1]:
        residuals = np.linalg.norm(A @ X - b)**2
    else:
        residuals = np.array([])  # Empty if no residuals are computed

    return X, residuals, rank, s


In [5]:
# Example
A = np.array([[1, 1], [1, -1], [1, 0]])
B = np.array([2, 0, 1])

X, residuals, rank, singular_values = gelsd(A, B)

print("A:\n", A, "\nB:\n", )
print("X:", X)
print("Residuals:", residuals)
print("Rank:", rank)
print("Singular values:", singular_values)
A@X

A:
 [[ 1  1]
 [ 1 -1]
 [ 1  0]] 
B:

X: [1. 1.]
Residuals: 0.0
Rank: 2
Singular values: [1.73205081 1.41421356]


array([2., 0., 1.])

# Bonus task 15: gelsy
Implement `gelsy` (with numpy, python and scipy.qr).

The routine first computes a QR factorization with column pivoting:
```
     A * P = Q * [ R11 R12 ]
                 [  0  R22 ]
```
 with R11 defined as the largest leading submatrix whose estimated
 condition number is less than 1/RCOND.  The order of R11, RANK,
 is the effective rank of A.

 Then, R22 is considered to be negligible, and R12 is annihilated
 by orthogonal transformations from the right, arriving at the
 complete orthogonal factorization:
 ```
    A * P = Q * [ T11 0 ] * Z
                [  0  0 ]
```
 The minimum-norm solution is then
```
    X = P * Z**T [ inv(T11)*Q1**T*B ]
                 [        0         ]
```
 where Q1 consists of the first RANK columns of Q.

In [6]:
def gelsy(A, B, rcond=1e-10):
    
    # Compute QR factorization with column pivoting
    Q, R, P = qr(A, pivoting=True, mode='economic')
    
    # Identify the effective rank
    diag_R = np.abs(np.diag(R))
    max_diag_R = np.max(diag_R)
    rank = np.sum(diag_R > rcond * max_diag_R)
    
    # Split R and Q based on the effective rank
    R11 = R[:rank, :rank]
    R12 = R[:rank, rank:]
    R22 = R[rank:, rank:]
    Q1 = Q[:, :rank]
    
    # Solve R11 * Y = Q1.T @ B
    QtB = Q1.T @ B
    Y = solve_triangular(R11, QtB[:rank], lower=False)
    
    # Reconstruct the solution using P
    X = np.zeros((A.shape[1],) if B.ndim == 1 else (A.shape[1], B.shape[1]))
    X[:rank] = Y
    X = X[np.argsort(P)]  # Apply the inverse permutation
    
    return X, rank


In [7]:
# Example
A = np.array([[2, 4, 1], [4, 6, 1], [2, 3, 10]], dtype=float)
B = np.array([11, 34, 21], dtype=float)

X, rank = gelsy(A, B)
print("X:", X)
print("Rank:", rank)

A@X

X: [17.71052632 -6.21052632  0.42105263]
Rank: 3


array([11., 34., 21.])

# Bonus task 16: Cholesky
Fix bugs in the Cholesky decomposition algorithm.

There are two bugs:
1. Example matrix a is not positive-definite
2. The algorithm attempts to access indices of the matrix L that do not exist

In [8]:
eps = 1.8e-17

a = np.array([[eps, 1], [1, 1.0]])

In [9]:
### THIS WORKS

def cholesky(a):

    try:
        # Check if the matrix is square
        n_rows, n_cols = a.shape
        if n_rows != n_cols:
            raise ValueError(f"Input matrix A must be square. Got {n_rows} rows and {n_cols} columns.")
        
        n = a.shape[0]
        
        # Check if the matrix is symmetric
        if not np.allclose(a, a.T, atol=1e-8):
            raise ValueError("Input matrix A must be symmetric.")
    
        # Fix the first bug
        # Check if the matrix positive-definite
        if not np.all(np.linalg.eigvals(a) > 0):
            raise ValueError("Matrix A is not positive-definite.")
    
        L = np.zeros_like(a)
        
        for i in range(n):
            for k in range(i + 1):
                s = 0.0
                for j in range(k):
                    s += L[i, j] * L[k, j]
                if i == k:
                    L[i, i] = np.sqrt(a[i, i] - s)
                else:
                    # Fix the secong bug
                    L[i, k] = (a[i, k] - s) / L[k, k]
    
        return L

    except ValueError as e:
            print(f"Error during the decomposition: {e}")
            return None

In [10]:
cholesky(a)

Error during the decomposition: Matrix A is not positive-definite.


In [11]:
a = np.array([[4, 12, -16],
              [12, 37, -43],
              [-16, -43, 98]], dtype=float)

L = cholesky(a)
print(f"My Cholesky:\n{L}")
print(f"scipy Cholesky:\n{np.linalg.cholesky(a)}")

My Cholesky:
[[ 2.  0.  0.]
 [ 6.  1.  0.]
 [-8.  5.  3.]]
scipy Cholesky:
[[ 2.  0.  0.]
 [ 6.  1.  0.]
 [-8.  5.  3.]]


In [12]:
L @ L.T

array([[  4.,  12., -16.],
       [ 12.,  37., -43.],
       [-16., -43.,  98.]])